In [5]:
import sys
import math
import random
import itertools
import os
import numpy as np
import scipy.sparse.csgraph
import networkx as nx
from SetCoverPy import *
from gurobipy import *

In [6]:
INPUT_DIR = './inputs'
input0 = os.path.join(INPUT_DIR, '52.in')

In [7]:
def read_file(file):
    with open(file, 'r') as f:
        data = f.readlines()
    data = [line.strip().split() for line in data]
    return data

def data_parser(input_data):
    number_of_kingdoms = int(input_data[0][0])
    list_of_kingdom_names = input_data[1]
    starting_kingdom = input_data[2][0]
    adjacency_matrix = [[entry if entry == 'x' else float(entry) for entry in row] for row in input_data[3:]]
    return number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix

In [8]:
def matrix(input_file):
    adj = np.genfromtxt(input_file, dtype=float, skip_header=3)
    size = len(adj)
    for i in range(size):
        for j in range(size):
            if np.isnan(adj[i][j]):
                adj[i][j] = 0
    return adj

def preprocess(input_file):
    data = read_file(input_file)
    number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix = data_parser(data)
    adjacency_matrix = matrix(input_file)
    return number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix

number_of_kingdoms, list_of_kingdom_names, starting_kingdom, adjacency_matrix = preprocess(input0)

In [9]:
g = matrix(input0)
gnx =nx.from_numpy_matrix(g)
paths = list(nx.all_pairs_dijkstra_path(gnx))
recon = []
for p in paths:
    a = []
    for i in range(number_of_kingdoms):
        a.append(p[1][i])
    recon.append(a)
shortest_dist = scipy.sparse.csgraph.floyd_warshall(g)

array([[  0.  ,  24.1 ,   1.  , ...,  98.  ,  98.95,  98.  ],
       [ 24.1 ,   0.  ,  24.1 , ...,  74.1 ,  75.05,  74.1 ],
       [  1.  ,  24.1 ,   0.  , ...,  97.  ,  97.95,  97.  ],
       ..., 
       [ 98.  ,  74.1 ,  97.  , ...,   0.  ,   1.  ,   2.  ],
       [ 98.95,  75.05,  97.95, ...,   1.  ,   0.  ,   1.  ],
       [ 98.  ,  74.1 ,  97.  , ...,   2.  ,   1.  ,   0.  ]])

In [13]:
m = Model()
vars = {}
n = shortest_dist.shape[0] #number of kingdoms
#Add edges as variables
for i in range(n):
    for j in range(i+1):
        vars[i,j] = m.addVar(obj=shortest_dist[i,j], vtype=GRB.BINARY, name='e'+str(i)+'_'+str(j))
        vars[j,i] = vars[i,j]
    m.update()

<gurobi.Model MIP instance Unnamed: 0 constrs, 11325 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>
